<a href="https://colab.research.google.com/github/Bhoomika685/Garbage_Classificcation-using-EfficientNet/blob/main/Garbage_Classification_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q tensorflow matplotlib scikit-learn

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

In [7]:
import zipfile

zip_path = '/content/archive (1).zip'  # Change if your file has a different name
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/garbage_data')

In [8]:
'/content/garbage_data/Garbage classification/Garbage classification'

'/content/garbage_data/Garbage classification/Garbage classification'

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

In [10]:
train_gen = train_datagen.flow_from_directory(
    '/content/garbage_data/Garbage classification/Garbage classification',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    '/content/garbage_data/Garbage classification/Garbage classification',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [12]:
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

base_model = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # freeze base model initially

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks again
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint("fine_tuned_model.h5", save_best_only=True)

# Train for 3–5 epochs
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=[early_stop, checkpoint]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.1408 - loss: 1.8002 

64/64 ━━━━━━━━━━━━━━━━━━━━ 1813s 27s/step - accuracy: 0.1411 - loss: 1.8001 - val_accuracy: 0.2068 - val_loss: 1.7867
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.2268 - loss: 1.7809 

64/64 ━━━━━━━━━━━━━━━━━━━━ 1694s 27s/step - accuracy: 0.2268 - loss: 1.7808 - val_accuracy: 0.3062 - val_loss: 1.7697
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.2310 - loss: 1.7674 

64/64 ━━━━━━━━━━━━━━━━━━━━ 1675s 26s/step - accuracy: 0.2311 - loss: 1.7673 - val_accuracy: 0.2823 - val_loss: 1.7529
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.2685 - loss: 1.7492 

64/64 ━━━━━━━━━━━━━━━━━━━━ 1678s 26s/step - accuracy: 0.2685 - loss: 1.7491 - val_accuracy: 0.2982 - val_loss: 1.7292
Epoch 5/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.2915 - loss: 1.7214 

64/64 ━━━━━━━━━━━━━━━━━━━━ 1706s 26s/step - accuracy: 0.2914 - loss: 1.7214 - val_accuracy: 0.3241 - val_loss: 1.7014


In [18]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load image and preprocess
img_path = '/content/paper.jpeg'  # Change to your image name
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)
predicted_class = np.argmax(pred)
confidence = np.max(pred)

# Get class labels
class_labels = list(train_gen.class_indices.keys())

print(f"🧠 Predicted Class: {class_labels[predicted_class]}")
print(f"📊 Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step
🧠 Predicted Class: paper
📊 Confidence: 0.24


In [17]:
class_labels = list(train_gen.class_indices.keys())
print(class_labels)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
